### Notebook creado por **Guillermo Grande Santi**

# Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import math
from collections import Counter
# from scipy.stats import norm

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_val_score
import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import LSTM, Dense # type: ignore
from tensorflow.keras.preprocessing.sequence import pad_sequences # type: ignore
from tensorflow.keras.preprocessing.text import Tokenizer # type: ignore

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.backends.cudnn as cudnn

# from sentence_transformers import SentenceTransformer
# from transformers import AutoTokenizer, AutoModel
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import nltk
import re
import string
import spacy
import contractions

import shap

2025-04-30 19:36:50.678836: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746034610.743306   24554 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746034610.771094   24554 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1746034610.928675   24554 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746034610.928703   24554 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1746034610.928704   24554 computation_placer.cc:177] computation placer alr

# Explicabilidad

### Modelo con Tensorflow para facilitar explicabilidad

En un inicio, no se ha empleado **TensorFlow** para entrenar los modelos por dos motivos principales:

1. Empleamos TensorFlow 2.19.0 junto con CUDA 12.1, pero a partir de la versión 2.11 el paquete oficial para Windows dejó de incluir **soporte GPU** (el último que lo integra de forma nativa es el 2.10, que requiere CUDA 11.2).

2. Durante el Máster, en asignaturas como *Redes Neuronales* y *Aprendizaje Profundo*, trabajamos habitualmente con **PyTorch**, lo cual resultó más ágil y familiar y se prefirió por encima de cambiar versiones de CUDA y Tensorflow.

Sin embargo, muchos métodos de explicabilidad están diseñados para TensorFlow y la definición y el entrenamiento de modelos en Keras suelen ser más sencillos. Por ello, a continuación migraremos nuestro mejor modelo de PyTorch a TensorFlow. Además, hemos levantado una máquina virtual Debian bajo WSL2 para aprovechar la GPU y reducir drásticamente los tiempos de entrenamiento.

In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Cargar el DataFrame limpio
df = pd.read_csv("Datasets/Cleaned-FR-News_V2.csv")

# Dividimos los datos en entrenamiento y prueba
# Por ahora usaremos únicamente el texto de la noticia (omitimos el título)
X = df["clean_text"]
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Se usará para redes neuronales
# Usaremos un 20% del conjunto de datos para validación (16% del total)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

def create_dataset(train, test, shuffle=True):
    # Create a TensorFlow dataset from the text and fake columns of the dataframe
    dataset = tf.data.Dataset.from_tensor_slices((train, test))
    if shuffle:
         # Shuffle the dataset if the shuffle parameter is True
        dataset = dataset.shuffle(1024, reshuffle_each_iteration=True)
    # Batch the dataset into smaller batches of size 256
    dataset = dataset.batch(256).cache().prefetch(tf.data.AUTOTUNE)
    # Prefetch the next batch of data to further optimize training
    return dataset

train_ds = create_dataset(X_train, y_train)
valid_ds = create_dataset(X_valid, y_valid, shuffle=False)
test_ds = create_dataset(X_test, y_test, shuffle=False)

I0000 00:00:1746034630.794990   24554 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5592 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Ti, pci bus id: 0000:05:00.0, compute capability: 8.6


In [5]:
# Create a TextVectorization layer with specified parameters
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=10000, 
    output_sequence_length=1024, 
    pad_to_max_tokens=True
)
# Adapt the TextVectorization layer to the training data
vectorizer.adapt(X_train, batch_size=1024)

2025-04-30 19:37:13.619942: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 104327280 exceeds 10% of free system memory.
2025-04-30 19:37:13.620009: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 156490920 exceeds 10% of free system memory.


In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(), dtype=tf.string),
    vectorizer,
    tf.keras.layers.Embedding(
        input_dim=10000, 
        output_dim=64,
        input_length=1024, 
        mask_zero=True
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)), 
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)), 
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(), 
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
    metrics=[
        "accuracy", 
        tf.keras.metrics.AUC(name="auc")
    ]
)
model.summary()
tf.keras.utils.plot_model(model)

/home/ggs/Fake-News-Detection/.venv/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, 1024)           │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 1024, 64)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 1024, 128)      │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 748,321 (2.85 MB)

 Trainable params: 748,321 (2.85 MB)

 Non-trainable params: 0 (0.00 B)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [7]:
# Verificación rápida
import tensorflow as tf
print("Asignación de ops en dispositivos:")
tf.debugging.set_log_device_placement(True)

file_path = "models/best_bilstm.keras"
history = model.fit(
    train_ds,
    epochs=10, 
    validation_data=valid_ds,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            file_path,
            save_best_only=True,
            monitor='val_accuracy',
            mode='max'
        )
    ]
)

Asignación de ops en dispositivos:
Epoch 1/10


I0000 00:00:1746034650.677802   24673 cuda_dnn.cc:529] Loaded cuDNN version 90501


111/111 ━━━━━━━━━━━━━━━━━━━━ 46s 375ms/step - accuracy: 0.8834 - auc: 0.9362 - loss: 0.3759 - val_accuracy: 0.9727 - val_auc: 0.9963 - val_loss: 0.0867
Epoch 2/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 42s 376ms/step - accuracy: 0.9831 - auc: 0.9980 - loss: 0.0555 - val_accuracy: 0.9877 - val_auc: 0.9985 - val_loss: 0.0396
Epoch 3/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 41s 366ms/step - accuracy: 0.9954 - auc: 0.9996 - loss: 0.0163 - val_accuracy: 0.9884 - val_auc: 0.9979 - val_loss: 0.0415
Epoch 4/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 40s 360ms/step - accuracy: 0.9987 - auc: 0.9999 - loss: 0.0070 - val_accuracy: 0.9884 - val_auc: 0.9966 - val_loss: 0.0445
Epoch 5/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 40s 362ms/step - accuracy: 0.9992 - auc: 0.9999 - loss: 0.0047 - val_accuracy: 0.9868 - val_auc: 0.9974 - val_loss: 0.0415
Epoch 6/10
111/111 ━━━━━━━━━━━━━━━━━━━━ 41s 371ms/step - accuracy: 0.9997 - auc: 0.9999 - loss: 0.0028 - val_accuracy: 0.9884 - val_auc: 0.9981 - val_loss: 0.0368
Epoch 7/10
111/111 ━━━━━━━━━━━━━━

Consideramos óptimo el modelo que alcanza el máximo valor de *val_accuracy* durante el entrenamiento. En este experimento, la mejor convergencia se produce en la segunda época, con un val_accuracy del **99,85 %** y una pérdida de **0,0396**, lo que supone un ligero progreso respecto al modelo en PyTorch. A continuación, presentamos los resultados obtenidos sobre el conjunto de prueba.

In [ ]:
# Evaluate the TensorFlow model using the test_ds dataset
model = tf.keras.models.load_model(file_path)
eval_results = model.evaluate(test_ds)


35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 112ms/step - accuracy: 0.9911 - auc: 0.9986 - loss: 0.0295


En el conjunto de prueba, el modelo alcanza una precisión de **99,11 %**, el mejor resultado obtenido hasta la fecha. El área bajo la curva ROC —que mide la capacidad del modelo para distinguir correctamente entre clases positivas y negativas en todos los umbrales posibles— es de **99,86 %**, y la pérdida final se sitúa en **0,0295**.

Es posible que la pequeña mejora (de aproximadamente un **0,2 %** respecto al modelo en PyTorch) se deba a una mejor semilla de inicialización y a utilizar un tamaño de *batch* mayor, que en este caso fue de 256.




### Explicabilidad mediante Deep SHAP

El método **Deep Explainer** de SHAP no puede utilizar los 28.000 datos de entrenamiento completos debido a las limitaciones computacionales al calcular los **valores SHAP para cada muestra** en un modelo complejo como **BiLSTM** (El proceso requiere evaluar el modelo en múltiples combinaciones de características).

Por esta razón, se seleccionan **muestras de fondo** (background samples), para aproximar el valor esperado de la salida del modelo (E[f(x)]). La aproximación se basa en la idea de que la diferencia entre la salida del modelo para una muestra específica f(x) y el valor esperado E[f(x)] (promedio de las salidas sobre las muestras de fondo) refleja cómo una característica en particular contribuye a la predicción para esa muestra. El uso de estas muestras permite realizar estimaciones eficientes de los valores SHAP **sin sacrificar la calidad de las explicaciones**. 

### Explicabilidad mediante Integrated Gradients